In [1]:
# @formatter:off
import random

import numpy as np
%load_ext autoreload
%autoreload 2
# @formatter:on
import os

os.chdir('../')
print(os.getcwd())

/home/viktor/workspace/smaite-scripts


In [2]:
## adjust this accordingly
#@formatter:off
files = !ls data/processed/*.jsonl
#@formatter:on
print(files)

['data/processed/len1001-1500-batch1.jsonl', 'data/processed/len1001-1500-batch2.jsonl', 'data/processed/len1001-1500-batch3.jsonl', 'data/processed/len1501-2000-batch1.jsonl', 'data/processed/len1501-2000-batch2.jsonl', 'data/processed/len1501-2000-batch3.jsonl', 'data/processed/len1501-2000-batch4.jsonl']


In [3]:
from handystuff.loaders import load_jsonl
data = []
for f in files:
    data.extend(load_jsonl(f))

In [4]:
len(data)

2621

In [5]:
chatgpt_answers = load_jsonl('chatgpt0-2621.jsonl')

In [6]:
assert len(data) == len(chatgpt_answers)

In [7]:
binary = ['contra-article_1', 'contra-self_1', 'convince_1', 'new_1', 'verdict_1', 'simplified_verdict_1']
ordinal = ['rate_1']

In [8]:
ordinal_scale_mapping = {
    'false': 'false',
    'true': 'true',
    'satire': 'false',
    'unk': 'half',
    'half': 'half',
    'hardly': 'half',
    'almost': 'half',
    'false': 'false',
    -1: -1,
    False: 'false',
    True: 'true',
    'inbetween': 'half',
}

In [9]:
simplified_ordinal_scale_mapping = {
    'false': 'false',
    'true': 'true',
    'satire': 'false',
    'unk': 'inbetween',
    'half': 'inbetween',
    'hardly': 'inbetween',
    'almost': 'inbetween',
    -1: -1,
    False: 'false',
    True: 'true',
    'inbetween': 'inbetween',
}

In [10]:
print(set(a['verdict_1'] for d in data for a in d['answers']))

{'almost', 'half', 'true', 'satire', 'unk', 'false', -1, 'hardly'}


In [11]:
from typing import Any, List


def how_many_equal(values: List[Any]):
    max_agreements = max(sum(k == values[i] for j, k in enumerate(values) if i != j) + 1 for i in range(len(values)))
    return max_agreements if max_agreements > 1 else 0

In [12]:
import json
failed = 0
for chatgpt_answer, d in zip(chatgpt_answers, data):
    try:
        answer = json.loads(chatgpt_answer[:chatgpt_answer.index('}')+1])
    except:
        failed += 1
        print(chatgpt_answer)
        continue
    answer = {f"{k}_1":v for k,v in answer.items()}
    #answer['verdict'] = simplified_ordinal_scale_mapping[answer['verdict']]
    answer['worker'] = 'CHATGPT'
    d['answers'].append(answer)
print(failed)

ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee mee mee content policy bleurgh
ChatGPT mee

In [13]:
for d in data:
    for a in d['answers']:
        a['simplified_verdict_1'] = ordinal_scale_mapping[a['verdict_1']]

In [14]:
data[0]

{'claim': 'Photograph of people filling the streets of Paris was taken during protests against Covid-19 regulations on 17 July 2021.',
 'verdict': 'This photograph was taken during celebrations for France’s World Cup win in 2018.',
 'text': "A post on Facebook shared almost 1,000 times shows people lining the streets of Paris. The caption of the post claims this photograph was taken on 17 July 2021, when there were protests in the French capital over Covid-19 rules. However, this photograph actually shows celebrations in the city after France won the World Cup in 2018. \nSince this fact check was published, the Facebook post has been updated to show a different image of people protesting against Covid-19 regulations.\nOn 12 July, French president Emmanuel Macron announced new measures against Covid-19, including the introduction of Covid-19 'passports' and that vaccination will become mandatory for all health workers. \nThe planned new measures have led to large protests in the country

In [15]:
from tqdm import tqdm
def calculate_data_agreement(data, key_suffix=''):
    for d in tqdm(data):
        d[f'agreements{key_suffix}'] = dict()
        for answers in zip(*[a.items() for a in d['answers']]):
            keys, values = zip(*answers)
            values = [v for v in values if v != -1]
            k = f"{keys[0]}{key_suffix}"
            if k in ordinal:
                d[k] = sum(values)/len(values)
            elif k in binary:
                if k == 'verdict_1':
                    agreement = how_many_equal([ordinal_scale_mapping[v] for v in values])
                else:
                    agreement = how_many_equal(values)
                d['agreements'][k] = agreement/len(values)
                d[k] = max(values, key=values.count) if agreement > 0.5 else 'NO_AGREEMENT'
calculate_data_agreement(data)

100%|██████████| 2621/2621 [00:00<00:00, 37154.24it/s]


In [16]:
data[1025]

{'claim': 'We spend nearly £50 billion on debt interest in the UK.',
 'verdict': 'This is not correct. The government actually pays interest on its debts to itself.',
 'text': '"Today we spend nearly PS50 billion in payments on interest to those we have borrowed from as a result of the legacy of the Labour party. That is more than the NHS pay bill, it is more than our schools budget, more than we spend on defence, as a result of the record we were left by Labour in government"\nTheresa May, 1 November 2017\nThe UK government\'s debt interest bill was about PS48 billion in 2016/17, but that\'s not the most meaningful figure to be using.\nThe government effectively pays some of that to itself. As we\'ve covered before, almost a quarter of government debt is owned by the Bank of England, which pays the profit back to the Treasury.\nThe total amount of spending on what are called \'public debt transactions\' amounted to PS39 billion in 2016/17, once you take off what the Bank of England ow

In [17]:
for k in ['contra-article_1', 'contra-self_1', 'verdict_1', 'convince_1', 'new_1', 'simplified_verdict_1']:
    percent_perfect_agreement = sum(d['agreements'][k] == 1.0 for d in data)/len(data)
    print(f"{k} perfect agreement: {percent_perfect_agreement:.2f}")
print('----'*20)
for k in ['contra-article_1', 'contra-self_1', 'verdict_1', 'convince_1', 'new_1', 'simplified_verdict_1']:
    percent_at_least_some_agreement = sum(d['agreements'][k] > 0.5 for d in data)/len(data)
    print(f"{k} some agreement: {percent_at_least_some_agreement:.2f}")

contra-article_1 perfect agreement: 0.20
contra-self_1 perfect agreement: 0.24
verdict_1 perfect agreement: 0.11
convince_1 perfect agreement: 0.17
new_1 perfect agreement: 0.12
simplified_verdict_1 perfect agreement: 0.11
--------------------------------------------------------------------------------
contra-article_1 some agreement: 0.72
contra-self_1 some agreement: 0.74
verdict_1 some agreement: 0.53
convince_1 some agreement: 0.66
new_1 some agreement: 0.64
simplified_verdict_1 some agreement: 0.53


In [18]:
from collections import defaultdict
keys = binary + ordinal
def get_agreement_per_author(data):
    per_key = {k: defaultdict(list) for k in keys}
    binary_only = defaultdict(list)
    overall = defaultdict(list)
    for d in tqdm(data):
        for a in d['answers']:
            worker = a['worker']
            for k in keys:
                answer = a[k]
                other_answers = [oa[k] for oa in d['answers'] if oa['worker'] != worker and oa[k] != -1]
                if k == 'verdict_1':
                    answer = ordinal_scale_mapping[answer]
                    other_answers = [ordinal_scale_mapping[a] for a in other_answers]
                # print(d['answers'])
                # print(a)
                # print(k)
                # print(answer)
                # print(other_answers)
                # print(int(any(answer == oa for oa in other_answers)))
                if answer != -1:
                    score = any(answer == oa for oa in other_answers)
                    per_key[k][worker].append(score)
                    overall[worker].append(score)
                    if k in binary and k != 'verdict_1' and k!='simplified_verdict_1':
                        binary_only[worker].append(score)
                    
    return per_key, overall, binary_only
per_key, overall, binary_only = get_agreement_per_author(data)

100%|██████████| 2621/2621 [00:00<00:00, 30244.75it/s]


In [19]:
overall_percentage = {worker: sum(v)/len(v) for worker, v in overall.items()}
binary_percentage = {worker: sum(v)/len(v) for worker, v in binary_only.items()}

In [20]:
by_key_percentage = {key: {worker: sum(v)/len(v) for worker, v in value.items()} for key, value in per_key.items()}

In [21]:
for k, v in by_key_percentage.items():
    print(k)
    print(v)

contra-article_1
{'A3MMMP508IF3M7': 0.8064516129032258, 'A3O0NNIW4KA2PV': 0.9647058823529412, 'A3RDJ3UZ9EJP6Y': 0.9616161616161616, 'CHATGPT': 0.9131832797427653, 'A2CFI1X8OKYIL2': 0.9567901234567902, 'A15SB9KAGZMJ4G': 0.821301775147929, 'A9W3MMLGGWVT1': 0.7963752665245203, 'A3V6FJ6TQCYA4E': 0.9439024390243902, 'A1R5JNC2LO6X1S': 0.90625, 'AKQAI78JTXXC9': 0.9090909090909091, 'A24AUZUNEH9LC3': 0.7631578947368421, 'A2VO8C41JJIQY9': 1.0, 'A146QN6EU8LAEI': 1.0, 'A1J980QNBDG17Z': 0.9700214132762313, 'A2QYXVJ6EKJBE6': 0.7707006369426752, 'A3GRE25RPUZ1QG': 0.9588607594936709, 'A3HSA4XVW7TNW7': 0.65, 'AQCZJ81IS07OK': 0.825, 'A3VDG4N48AVYGQ': 1.0, 'A2EJ7U3TZAKROG': 1.0, 'AX8NXTT8QMGHC': 0.8571428571428571, 'A1JG342PWI2PTC': 1.0, 'A3GI28OVLDI9ST': 1.0, 'A16QH3MFZ9CC4M': 0.8673835125448028, 'AXOJYPR75TD9X': 0.7938144329896907, 'A15B4KZ3S04HS8': 0.8530465949820788, 'AS787OL64GYWX': 0.8471337579617835, 'A367K74P9QF27V': 0.8826979472140762, 'ALWX7RRX16LEZ': 0.8625954198473282, 'ACF6USGTSNNXC': 0.7824

In [22]:
print(overall_percentage)

{'A3MMMP508IF3M7': 0.7674101821896816, 'A3O0NNIW4KA2PV': 0.8521008403361344, 'A3RDJ3UZ9EJP6Y': 0.833670812843506, 'CHATGPT': 0.848242994947175, 'A2CFI1X8OKYIL2': 0.8698328935795955, 'A15SB9KAGZMJ4G': 0.7588304884231875, 'A9W3MMLGGWVT1': 0.7522083460249771, 'A3V6FJ6TQCYA4E': 0.8435540069686411, 'A1R5JNC2LO6X1S': 0.8660714285714286, 'AKQAI78JTXXC9': 0.8701298701298701, 'A24AUZUNEH9LC3': 0.7849056603773585, 'A2VO8C41JJIQY9': 0.7142857142857143, 'A146QN6EU8LAEI': 0.9285714285714286, 'A1J980QNBDG17Z': 0.7821963903334354, 'A2QYXVJ6EKJBE6': 0.7042766151046406, 'A3GRE25RPUZ1QG': 0.8571428571428571, 'A3HSA4XVW7TNW7': 0.7025510204081633, 'AQCZJ81IS07OK': 0.7357142857142858, 'A3VDG4N48AVYGQ': 0.8571428571428571, 'A2EJ7U3TZAKROG': 0.873015873015873, 'AX8NXTT8QMGHC': 0.7346938775510204, 'A1JG342PWI2PTC': 0.7857142857142857, 'A3GI28OVLDI9ST': 0.8571428571428571, 'A16QH3MFZ9CC4M': 0.7916026625704045, 'AXOJYPR75TD9X': 0.8092783505154639, 'A15B4KZ3S04HS8': 0.7813620071684588, 'AS787OL64GYWX': 0.7634212

In [23]:
for k, value in by_key_percentage.items():
    print(k)
    for k,v in sorted(value.items(), key=lambda x: x[1], reverse=True):
        print(f'{k}: {v:.2f}')
print('overall')
for k,v in sorted(overall_percentage.items(), key=lambda x: x[1], reverse=True):
    print(f'{k}: {v:.2f}')
print('binary only')
for k,v in sorted(binary_percentage.items(), key=lambda x: x[1], reverse=True):
    print(f'{k}: {v:.2f}')

contra-article_1
A2VO8C41JJIQY9: 1.00
A146QN6EU8LAEI: 1.00
A3VDG4N48AVYGQ: 1.00
A2EJ7U3TZAKROG: 1.00
A1JG342PWI2PTC: 1.00
A3GI28OVLDI9ST: 1.00
AHKAW23MOXRQP: 1.00
A28RL79ONUXTET: 1.00
A1J980QNBDG17Z: 0.97
A3O0NNIW4KA2PV: 0.96
A3RDJ3UZ9EJP6Y: 0.96
A3GRE25RPUZ1QG: 0.96
A2CFI1X8OKYIL2: 0.96
A3V6FJ6TQCYA4E: 0.94
AUI3R928P63B0: 0.93
CHATGPT: 0.91
AKQAI78JTXXC9: 0.91
A1R5JNC2LO6X1S: 0.91
ABBJFGQWQC19S: 0.90
A367K74P9QF27V: 0.88
A18Z7Y10ENQDKY: 0.88
A16QH3MFZ9CC4M: 0.87
ALWX7RRX16LEZ: 0.86
AX8NXTT8QMGHC: 0.86
A15B4KZ3S04HS8: 0.85
AS787OL64GYWX: 0.85
A4ANRSA55IW5Q: 0.84
A1GH6S7TZ9083V: 0.83
AQCZJ81IS07OK: 0.82
A15SB9KAGZMJ4G: 0.82
A3MMMP508IF3M7: 0.81
A9W3MMLGGWVT1: 0.80
AXOJYPR75TD9X: 0.79
ACF6USGTSNNXC: 0.78
A2QYXVJ6EKJBE6: 0.77
A24AUZUNEH9LC3: 0.76
A1ODU7S7ENNY66: 0.75
A2DVDC9AM2E9ZG: 0.75
A2LTZWHXN5LDAT: 0.73
A3HSA4XVW7TNW7: 0.65
A3GC8HN0MYYY4: 0.50
A1IDP5YMUOTMH1: 0.00
contra-self_1
A3O0NNIW4KA2PV: 1.00
AKQAI78JTXXC9: 1.00
A2VO8C41JJIQY9: 1.00
A146QN6EU8LAEI: 1.00
A3VDG4N48AVYGQ: 1.00
A2E

In [24]:
data[0]['answers']

[{'contra-article_1': False,
  'contra-self_1': True,
  'rate_1': 3,
  'verdict_1': 'almost',
  'convince_1': True,
  'new_1': False,
  'worker': 'A3MMMP508IF3M7',
  'simplified_verdict_1': 'half'},
 {'contra-article_1': False,
  'contra-self_1': False,
  'rate_1': 4,
  'verdict_1': 'false',
  'convince_1': True,
  'new_1': False,
  'worker': 'A3O0NNIW4KA2PV',
  'simplified_verdict_1': 'false'},
 {'contra-article_1': False,
  'contra-self_1': False,
  'rate_1': 4,
  'verdict_1': 'true',
  'convince_1': True,
  'new_1': True,
  'worker': 'A3RDJ3UZ9EJP6Y',
  'simplified_verdict_1': 'true'},
 {'contra-article_1': True,
  'contra-self_1': False,
  'rate_1': 4,
  'verdict_1': False,
  'convince_1': True,
  'new_1': False,
  'worker': 'CHATGPT',
  'simplified_verdict_1': 'false'}]

In [25]:
def calculate_data_agreement_reliability(data, reliability_dict, key_suffix='', min_reliability=0.5, by_key=False):
    key_suffix = key_suffix or f'@{min_reliability}{"_bk" if by_key else ""}'
    for d in tqdm(data):
        d[f'agreements{key_suffix}'] = dict()
        all_answers = [item for item in zip(*[a.items() for a in d['answers']]) if item[0][0] != 'worker']
        workers = [a['worker'] for a in d['answers']]
        #print(list(all_answers))
        for answers in all_answers:
            # print(d['answers'])
            keys, values = zip(*answers)
            # print(keys)
            # print(values)
            # print(workers)
            original_k = keys[0]
            k = f"{keys[0]}{key_suffix}"
            if by_key:
                values = [v for v,w in zip(values, workers) if v != -1 and reliability_dict[original_k][w] >= min_reliability]
            else:
                values = [v for v,w in zip(values, workers) if v != -1 and reliability_dict[w] >= min_reliability]
            if values:                
                #print(k)
                if original_k in ordinal:
                    d[k] = sum(values)/len(values)
                elif original_k in binary:
                    if original_k == 'verdict_1':
                        agreement = how_many_equal([ordinal_scale_mapping[v] for v in values])
                    else:
                        agreement = how_many_equal(values)
                    d[f'agreements{key_suffix}'][k] = agreement/len(values)
                    #print(k)
                    d[k] = max(values, key=values.count) if agreement > 0.5 else 'NO_AGREEMENT'
            else:
                d[k] = None
calculate_data_agreement_reliability(data, overall_percentage, min_reliability=0.5, by_key=False)

100%|██████████| 2621/2621 [00:00<00:00, 29076.57it/s]


In [26]:
calculate_data_agreement_reliability(data, by_key_percentage, min_reliability=0.5, by_key=True)

100%|██████████| 2621/2621 [00:00<00:00, 30008.05it/s]


In [27]:
calculate_data_agreement_reliability(data, overall_percentage, min_reliability=0.75)

100%|██████████| 2621/2621 [00:00<00:00, 33640.07it/s]


In [28]:
calculate_data_agreement_reliability(data, by_key_percentage, min_reliability=0.75, by_key=True)

100%|██████████| 2621/2621 [00:00<00:00, 32055.02it/s]


In [29]:
calculate_data_agreement_reliability(data, overall_percentage, min_reliability=0.69, by_key=False)

100%|██████████| 2621/2621 [00:00<00:00, 31694.05it/s]


In [30]:
calculate_data_agreement_reliability(data, by_key_percentage, min_reliability=0.69, by_key=True)

100%|██████████| 2621/2621 [00:00<00:00, 31178.62it/s]


In [31]:
data[0].keys()

dict_keys(['claim', 'verdict', 'text', 'sources', 'answers', 'hit_id', 'agreements', 'contra-article_1', 'contra-self_1', 'rate_1', 'verdict_1', 'convince_1', 'new_1', 'simplified_verdict_1', 'agreements@0.5', 'contra-article_1@0.5', 'contra-self_1@0.5', 'rate_1@0.5', 'verdict_1@0.5', 'convince_1@0.5', 'new_1@0.5', 'simplified_verdict_1@0.5', 'agreements@0.5_bk', 'contra-article_1@0.5_bk', 'contra-self_1@0.5_bk', 'rate_1@0.5_bk', 'verdict_1@0.5_bk', 'convince_1@0.5_bk', 'new_1@0.5_bk', 'simplified_verdict_1@0.5_bk', 'agreements@0.75', 'contra-article_1@0.75', 'contra-self_1@0.75', 'rate_1@0.75', 'verdict_1@0.75', 'convince_1@0.75', 'new_1@0.75', 'simplified_verdict_1@0.75', 'agreements@0.75_bk', 'contra-article_1@0.75_bk', 'contra-self_1@0.75_bk', 'rate_1@0.75_bk', 'verdict_1@0.75_bk', 'convince_1@0.75_bk', 'new_1@0.75_bk', 'simplified_verdict_1@0.75_bk', 'agreements@0.69', 'contra-article_1@0.69', 'contra-self_1@0.69', 'rate_1@0.69', 'verdict_1@0.69', 'convince_1@0.69', 'new_1@0.69', 

In [37]:
for p in 'simplified_', '':
    for k in ('0.5', '0.5_bk', '0.69', '0.69_bk', '0.75', '0.75_bk'):
        for d in data:
            d[f'{p}plausible@{k}'] = True if d[f'{p}verdict_1@{k}'] != 'NO_AGREEMENT' else False

In [40]:
for k in 'contra-article_1', 'contra-self_1', 'rate_1', 'verdict_1', 'convince_1', 'new_1', 'plausible', 'simplified_verdict_1', 'simplified_plausible': 
    for ts in [None, 0.5, 0.69, 0.75]:
        for bk in ['','_bk']:
            full_key = f"{k}@{ts}{bk}" if ts else k
            label_dict = defaultdict(int)
            for d in data:
                if full_key not in d:
                    label_dict[None] += 1
                else:
                    label_dict[d[full_key]] += 1
            print(full_key, dict(label_dict))

contra-article_1 {False: 1885, True: 736}
contra-article_1 {False: 1885, True: 736}
contra-article_1@0.5 {False: 1885, True: 736}
contra-article_1@0.5_bk {False: 1885, True: 736}
contra-article_1@0.69 {False: 1885, True: 736}
contra-article_1@0.69_bk {False: 1948, True: 670, 'NO_AGREEMENT': 3}
contra-article_1@0.75 {False: 1965, True: 616, 'NO_AGREEMENT': 40}
contra-article_1@0.75_bk {False: 1959, True: 650, 'NO_AGREEMENT': 12}
contra-self_1 {False: 2037, True: 583, 'NO_AGREEMENT': 1}
contra-self_1 {False: 2037, True: 583, 'NO_AGREEMENT': 1}
contra-self_1@0.5 {False: 2037, True: 583, 'NO_AGREEMENT': 1}
contra-self_1@0.5_bk {False: 2037, True: 583, 'NO_AGREEMENT': 1}
contra-self_1@0.69 {False: 2037, True: 583, 'NO_AGREEMENT': 1}
contra-self_1@0.69_bk {False: 2123, True: 480, 'NO_AGREEMENT': 18}
contra-self_1@0.75 {False: 2137, True: 444, 'NO_AGREEMENT': 40}
contra-self_1@0.75_bk {False: 2136, True: 454, 'NO_AGREEMENT': 31}
rate_1 {3.75: 408, 4.25: 364, 3.5: 352, 4.0: 417, 3.333333333333

In [39]:
data[1234]

{'claim': 'An underground tunnel is a secret UK military site.',
 'verdict': 'This is not true. The site is a government-owned facility, and is closed to the public.',
 'text': 'A video on Facebook, viewed over 400,000 times, shows a group of people setting off an alarm in what appears to be an underground tunnel. \nThe caption says: "UK Area 51! What are they hiding down here!!"\nThis is a reference to a secret US Air Force site in Nevada, US. The site has been the focus of a number of conspiracy theories relating to extraterrestrial life, and is completely closed to the public.\nWhile the location of the video isn\'t made clear in the video, the account which originally posted the video to TikTok in December 2021 says in a number of comments that it was filmed at an underground Ministry of Defence (MoD) facility in Corsham, Wiltshire. \nFar from being a secret military site like Area 51, Corsham is home to a sprawling set of underground bunkers known by a number of names including th

In [42]:
from handystuff.loaders import write_jsonl

In [76]:
def we_need(k):
    return k in ('claim', 'verdict', 'sources', 'text') or any(k.startswith(p) for p in ('contra', 'verdict', 'convince', 'new', 'simplified_verdict', 'rate', 'plausible', 'simplified_plausible'))
def parse(v, k):
    if isinstance(v, list):
        return '\n'.join(lv for lv in v)
    if isinstance(v, float):
        return v
    if k.startswith('rate') and v is None:
        return 0
    else:
        return str(v)

In [77]:
parsed_data = [{k:parse(v,k) for k,v in d.items() if we_need(k)} for d in data]

In [79]:
write_jsonl(parsed_data, 'data-main-no-answers.json')

In [78]:
set(pd['rate_1@0.69_bk'] for pd in parsed_data)

{0, 1.0, 2.0, 3.0, 3.5, 4.0, 4.5, 5.0}